<a href="https://colab.research.google.com/github/butyusya/project_lbacn/blob/main/crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/butyusya/project_lbacn.git

Cloning into 'project_lbacn'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [6]:
!pip install fake_useragent

In [ ]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import re

In [ ]:
ua = UserAgent()
headers = {'User-Agent': ua.random}
session = requests.session()

In [ ]:
def parse_episode(episode):
  block = {}
  info = episode.find_all('td')
  id = info[4].text.strip()
  block['ep_id'] = int(re.sub(r'\[\d+\]', '', id))
  block['ep_href'] = info[2].find('a').attrs['href']
  block['ep_title'] = info[2].find('a').text
  return block

In [ ]:
def parse_episode_info(block):
  url = 'https://miraculousladybug.fandom.com' + block['ep_href']
  req = session.get(url, headers={'User-Agent': ua.random})
  body = req.text
  text = body.split('Synopsis</span>')[2]
  first_div = text.split('<span class="mw-headline" id="Plot">Plot</span>')
  syn = first_div[0]
  porridge = BeautifulSoup(syn, 'html.parser')
  synopsis = porridge.find('p').text.strip()
  block['synopsis'] = re.sub(r'\[\d+\]', '', synopsis)
  other = first_div[1]
  second_div = other.split('<h2>')
  pl = second_div[0]
  porridge = BeautifulSoup(pl, 'html.parser')
  pl_pars = porridge.find_all('p')
  plot = ''
  for p in pl_pars:
    plot += p.text
  block['plot'] = re.sub(r'\[\d+\]', '', plot)
  url_scr = 'https://miraculousladybug.fandom.com' + block['ep_href']+ '/Transcript'
  req = session.get(url_scr, headers={'User-Agent': ua.random})
  body = req.text
  script = body.split('</td></tr></tbody></table></center>')[1].split('THE END')[0]
  porridge = BeautifulSoup(script, 'html.parser')
  block['script'] = porridge.text.replace('[ Theme Song ]', '').replace('[ Theme song ]', '').strip()
  return block

In [ ]:
def parse_season(table, blocks):
  episodes = table.find_all('tr')
  for n in range(1, len(episodes)):
    try:
      block = parse_episode(episodes[n])
    except Exception as e:
      print(1, e, i, n)
    try:
      parse_episode_info(block)
    except Exception as e:
      print(2, e, i, n)
    try:
      blocks.append(block)
    except Exception as e:
      print(3, e, i, n)
  return blocks

In [ ]:
blocks = []
for i in range(1, 6):
  url_one = 'https://miraculousladybug.fandom.com/wiki/Season_' + str(i)
  req = session.get(url_one, headers={'User-Agent': ua.random})
  body = req.text
  soup = BeautifulSoup(body, 'html.parser')
  tables = soup.find_all('table')
  table = tables[4]
  parse_season(table, blocks)

2 ('Connection broken: IncompleteRead(3911680 bytes read, 1032338 more expected)', IncompleteRead(3911680 bytes read, 1032338 more expected)) 4 3
2 ('Connection broken: IncompleteRead(3780608 bytes read, 505786 more expected)', IncompleteRead(3780608 bytes read, 505786 more expected)) 4 14
2 ('Connection broken: IncompleteRead(3219456 bytes read, 1662788 more expected)', IncompleteRead(3219456 bytes read, 1662788 more expected)) 4 21
2 ('Connection broken: IncompleteRead(2965504 bytes read, 1718991 more expected)', IncompleteRead(2965504 bytes read, 1718991 more expected)) 5 24


In [ ]:
new_table_query = """
CREATE TABLE episodes (
    ep_id INT,
    ep_href TEXT,
    ep_title TEXT,
    synopsis TEXT,
    plot TEXT,
    script TEXT,
    PRIMARY KEY (ep_id)
)
"""

In [ ]:
import sqlite3

con = sqlite3.connect('lbacn.db')
cur = con.cursor()
cur.execute(new_table_query)
con.commit()

In [ ]:
for block in blocks:
  cur.execute("INSERT INTO episodes VALUES (:ep_id, :ep_href, :ep_title, :synopsis, :plot, :script)", block)
con.commit()

In [ ]:
def get_image(block):
  url = 'https://miraculousladybug.fandom.com' + block['ep_href']
  req = session.get(url, headers={'User-Agent': ua.random})
  body = req.text
  porridge = BeautifulSoup(body, 'html.parser')
  image_info = porridge.find('figure', {'class': 'pi-item pi-image'})
  image_url = image_info.find('a').attrs['href']
  img_data = requests.get(image_url).content
  with open('project_lbacn/static/'+str(block['ep_id'])+'.png', 'wb') as handler:
    handler.write(img_data)

In [ ]:
for b in blocks:
  try:
    get_image(b)
  except Exception as e:
    print(b['ep_id'])